# Validation of ips

In [72]:
import concurrent.futures
import requests

def check_ip(ip):
    url = "http://www.google.com"  # URL to check connectivity
    proxies = {"http": f"http://{ip}", "https": f"https://{ip}"}
    try:
        response = requests.get(url, proxies=proxies, timeout=5)
        if response.status_code == 200:
            return ip
    except Exception as e:
        pass
    return None

valid_ips = []
invalid_ips = []
with open("ips.txt", "r") as file:
    ips = file.read().splitlines()

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(check_ip, ips)

for result in results:
    if result:
        valid_ips.append(result)
        print(result)
    else:
        invalid_ips.append(result)

with open("valid_ips.txt", "w") as file:
    file.write("\n".join(valid_ips))

print("Valid IPs saved to valid_ips.txt")



38.62.222.122:3128
38.62.223.223:3128
38.62.222.183:3128
38.62.223.126:3128
154.6.97.241:3128
154.6.97.23:3128
38.62.220.125:3128
38.62.220.238:3128
154.6.99.111:3128
38.62.223.181:3128
38.62.223.16:3128
38.62.220.143:3128
38.62.221.92:3128
154.6.99.123:3128
38.62.222.243:3128
38.62.222.42:3128
154.6.99.7:3128
154.6.98.210:3128
154.6.99.83:3128
38.62.223.71:3128
38.62.220.172:3128
38.62.222.210:3128
154.6.98.135:3128
154.6.98.35:3128
38.62.222.11:3128
154.6.97.208:3128
38.62.220.215:3128
38.62.222.218:3128
154.6.99.216:3128
154.6.99.99:3128
154.6.99.222:3128
154.6.97.106:3128
154.6.97.202:3128
38.62.223.89:3128
38.62.220.159:3128
154.6.96.207:3128
38.62.221.62:3128
38.62.222.241:3128
38.62.220.190:3128
38.62.223.230:3128
154.6.99.179:3128
154.6.99.92:3128
154.6.99.186:3128
154.6.96.110:3128
38.62.222.109:3128
38.62.222.2:3128
154.6.96.226:3128
38.62.223.98:3128
154.6.99.79:3128
154.6.98.201:3128
154.6.99.58:3128
38.62.223.199:3128
38.62.221.116:3128
38.62.222.108:3128
38.62.223.208:312

In [30]:
import time
import random
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from fuzzywuzzy import process


import json

In [66]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def scrape_amazon_products(url, num_pages):
    # Initialize web driver
    driver = webdriver.Chrome()
    driver.get(url)

    product_links = []

    try:
        # Loop through each page
        for page in range(num_pages):
            # Wait for the product containers to load
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[data-component-type="s-impression-counter"]')))
            
            # Parse the page HTML
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Extract product links
            product_containers = soup.find_all('div', {'data-component-type': 's-impression-counter'})
            for container in product_containers:
                link = container.find('a', class_='a-link-normal')['href']
                product_links.append(link)
            
            # Navigate to the next page if available
            next_page_link = driver.find_element_by_css_selector('.s-pagination-next')
            if next_page_link:
                next_page_link.click()
                time.sleep(3)  # Adding a delay to ensure the page loads completely

        return product_links

    finally:
        driver.quit()

# Example usage
url = 'https://www.amazon.com/s?i=fashion-mens-intl-ship&bbn=16225019011&rh=n%3A7141123011%2Cn%3A16225019011%2Cn%3A7147441011%2Cn%3A679255011&dc&ds=v1%3AqZ5%2BjoYA1PFo4198cZy2c%2FLatdT9b3vO5f4HSStG%2FAQ&qid=1708491762&rnid=7141123011&ref=sr_nr_n_3'
num_pages = 5  

product_links = scrape_amazon_products(url, num_pages)
print(product_links)


In [31]:
def get_url_for_reviews(driver):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Wait for reviews to load
    
    try:
        see_more_reviews_link = driver.find_element(By.CSS_SELECTOR,"a[data-hook='see-all-reviews-link-foot']")
        url_for_reviews = see_more_reviews_link.get_attribute('href')
        return url_for_reviews
    except Exception as e:
        print("Error while fetching URL for reviews:", e)
        return None

In [44]:

def extract_product_id(url):
    match = re.search(r'/dp/([A-Z0-9]+)', url)
    if match:
        return match.group(1)
    return None

def get_random_proxy(proxies):
    return random.choice(proxies)


In [72]:
def extract_image_urls(driver):
    time.sleep(2)  
    image_elements = driver.find_elements(By.CSS_SELECTOR, 'div#altImages img')  
    urls = [element.get_attribute('src') for element in image_elements]
    return urls

In [46]:
def fetch_reviews(driver,url_for_reviews):
    driver.get(url_for_reviews)
    time.sleep(3)
    reviews_data = []
    review_divs = driver.find_elements(By.XPATH, "//div[@data-hook='review']")
    for review_div in review_divs:
        rating_element = review_div.find_element(By.XPATH, ".//i[@data-hook='review-star-rating']")
        rating_span = rating_element.find_element(By.XPATH, ".//span[@class='a-icon-alt']")
        rating = rating_span.get_attribute("innerHTML").strip().split()[0]
        description_element = review_div.find_element(By.XPATH, ".//span[@data-hook='review-body']")
        description = description_element.text.strip()
        reviews_data.append({"Rating": rating, "Description": description})
    return reviews_data


In [47]:
def click_color_options(driver):
    color_data = {}
    time.sleep(1)
    color_options = driver.find_elements(By.CSS_SELECTOR, 'li[data-csa-c-item-type="asin"]')
    print("Color Option ",len(color_options))
    for option in color_options:
        option.click()
        time.sleep(2)  
        selected_color = option.get_attribute("title")
        selected_color = selected_color.replace("Click to select ", "")
        urls = extract_image_urls(driver)
        color_data[selected_color] = {"images": urls}

    return color_data

In [48]:
# Function to scrape data using BeautifulSoup
def scrape_with_bs(page_source):
    soup = BeautifulSoup(page_source, 'html.parser')

    # Extracting product title
    product_title = soup.find('span', id='productTitle').get_text(strip=True)

    # Extracting review rating
    review_rating = soup.find('span', id='acrPopover').find('span', class_='a-icon-alt').get_text(strip=True)
    review_rating = review_rating[:3]
    # Extracting brand name
    brand_name = soup.find('a', id='bylineInfo').get_text(strip=True).replace('Visit the ', '')

    return product_title, review_rating, brand_name



In [49]:
def get_subCategory(category, sku):
    if category == "Shoes":
        listCategory = ["Men's Athletic Shoes", "Men's Boots", "Men's Fashion Sneakers", 
                        "Men's Loafers & Slip-Ons", "Men's Mules & Clogs", "Men's Oxfords", 
                        "Men's Sandals", "Men's Slippers", "Men's Work & Safety Footwear"]
        match = process.extractOne(sku, listCategory)
        if match[1] >= 80:  
            return match[0]
        else:
            return "Subcategory not found"
    else:
        return "Category not found"

In [73]:
def get_product_description(driver):
    time.sleep(2)  
    try:
        description = driver.find_element(By.ID, "productDescription").text
    except:
        pass
    return ""

In [74]:
def solve_amazon_captcha(driver):
    captcha = AmazonCaptcha.fromdriver(driver)
    solution = captcha.solve()
    captcha_input = driver.find_element(By.ID, 'captchacharacters')
    captcha_input.send_keys(solution)
    time.sleep(2)  
    continue_button = driver.find_element(By.XPATH, "//span[@class='a-button-inner']/button[@type='submit']")
    continue_button.click()

In [83]:
def save_json_data(json_data, filename):
    with open(filename, 'a') as file:
        json.dump(json_data, file, indent=None)
        file.write('\n')  

In [80]:
def scrape_with_selenium(url, proxy):
    # Set up Selenium options
    chrome_options = Options()
    # chrome_options.add_argument("--headless")  # To run Chrome in headless mode
    chrome_options.add_argument(f"--proxy-server={proxy}")

    # Set up fake user agent
    ua = UserAgent()
    user_agent = ua.random
    chrome_options.add_argument(f'user-agent={user_agent}')

    # Initialize webdriver
    driver = webdriver.Chrome(options=chrome_options)

    try:
        driver.get(url)
        time.sleep(3)  
        if 'validateCaptcha' in driver.current_url:
            solve_amazon_captcha(driver)
        page_source = driver.page_source
        product_title, review_rating, brand_name = scrape_with_bs(page_source)
        color_data = click_color_options(driver)
        discription = get_product_description(driver)
        product_id = extract_product_id(url)
        reviewLink= get_url_for_reviews(driver)
        subCategory = get_subCategory("Shoes",product_title)
        print("Review Link: ",reviewLink)
        reviewData=fetch_reviews(driver,reviewLink)
        json_data = {
            "product_id": product_id,
            "product_title": product_title,
            "overall_rating": review_rating,
            "gender":"men",
            "category":"Shoes",
            "sub-category":subCategory,
            "discription":discription,
            "brand_name": brand_name,
            "colors": color_data,
            "reviews":reviewData,
            "reviewLink":reviewLink
        }
        print(json.dumps(json_data, indent=4))
        save_json_data(json_data, 'output.json')
    finally:
        pass
        driver.quit()


In [81]:
def main():
    with open('links.txt', 'r') as file:
        urls = file.read().splitlines()
    
    with open('valid_ips.txt', 'r') as file:
        proxies = file.read().splitlines()

    for url in urls:
        proxy = get_random_proxy(proxies)
        scrape_with_selenium(url, proxy)
        time.sleep(5)  

if __name__ == "__main__":
    main()

Color Option  33
Review Link:  https://www.amazon.com/Under-Armour-Charged-Assert-Running/product-reviews/B087P32XQG/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
{
    "product_id": "B087TPWKFJ",
    "product_title": "Under Armour Men's Charged Assert 9 Running Shoe",
    "overall_rating": "4.6",
    "gender": "men",
    "sub-category": "Men's Athletic Shoes",
    "discription": "",
    "brand_name": "Under Armour Store",
    "colors": {
        "Black-003": {
            "images": []
        },
        "Black/White-001": {
            "images": []
        },
        "Academy Blue (400)/White": {
            "images": []
        },
        "White/Black-100": {
            "images": []
        },
        "(009) Black/Blue Surf/White": {
            "images": []
        },
        "(119) Pitch Gray/Capri/Black": {
            "images": []
        },
        "Black (001)/Black": {
            "images": []
        },
        "Black (007)/Metallic Gold": {
            "image

Exception ignored in: <function Service.__del__ at 0x00000216A3A28C20>
Traceback (most recent call last):
  File "C:\Users\lenovo\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 189, in __del__
    self.stop()
  File "C:\Users\lenovo\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "C:\Users\lenovo\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 126, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "C:\Users\lenovo\anaconda3\Lib\urllib\request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lenovo\anaconda3\Lib\urllib\request.py", line 519, in open
    response = self._open(req, data)
               ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lenovo\anaconda3\Lib\urllib\request.py", line 536, in _open
    result = self._call_chain(self.handl

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=121.0.6167.185)
Stacktrace:
	GetHandleVerifier [0x00007FF74D037012+3522402]
	(No symbol) [0x00007FF74CC58352]
	(No symbol) [0x00007FF74CB05ABB]
	(No symbol) [0x00007FF74CAE287C]
	(No symbol) [0x00007FF74CB75D97]
	(No symbol) [0x00007FF74CB8B3CF]
	(No symbol) [0x00007FF74CB6EE03]
	(No symbol) [0x00007FF74CB3F4D4]
	(No symbol) [0x00007FF74CB405F1]
	GetHandleVerifier [0x00007FF74D069B9D+3730157]
	GetHandleVerifier [0x00007FF74D0BF02D+4079485]
	GetHandleVerifier [0x00007FF74D0B75D3+4048163]
	GetHandleVerifier [0x00007FF74CD8A649+718233]
	(No symbol) [0x00007FF74CC64A3F]
	(No symbol) [0x00007FF74CC5FA94]
	(No symbol) [0x00007FF74CC5FBC2]
	(No symbol) [0x00007FF74CC4F2E4]
	BaseThreadInitThunk [0x00007FFDAA6A257D+29]
	RtlUserThreadStart [0x00007FFDAB44AA58+40]


In [ ]:
def main():
    with open('links.txt', 'r') as file:
        urls = file.read().splitlines()
    
    with open('valid_ips.txt', 'r') as file:
        proxies = file.read().splitlines()

    for url in urls:
        proxy = get_random_proxy(proxies)
        scrape_with_selenium(url, proxy)
        time.sleep(5)  

if __name__ == "__main__":
    main()

Color Option  33
Review Link:  https://www.amazon.com/Under-Armour-Charged-Assert-Running/product-reviews/B087P32XQG/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews
{
    "product_id": "B087TPWKFJ",
    "product_title": "Under Armour Men's Charged Assert 9 Running Shoe",
    "overall_rating": "4.6",
    "gender": "men",
    "sub-category": "Men's Athletic Shoes",
    "discription": "",
    "brand_name": "Under Armour Store",
    "colors": {
        "Black-003": {
            "images": [
                "https://m.media-amazon.com/images/G/01/HomeCustomProduct/360_icon_73x73v2._CB485971279_SS40_FMpng_RI_.png",
                "https://m.media-amazon.com/images/I/31pKn-zkB2L._AC_US40_.jpg",
                "https://m.media-amazon.com/images/I/31ea7SIw3LL._AC_US40_.jpg",
                "https://m.media-amazon.com/images/I/31sIbobhvVL._AC_US40_.jpg",
                "https://m.media-amazon.com/images/I/31GtZld6odL._AC_US40_.jpg",
                "https://m.media-amazon.com/im

Exception ignored in: <function Service.__del__ at 0x00000216A3A28C20>
Traceback (most recent call last):
  File "C:\Users\lenovo\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 189, in __del__
    self.stop()
  File "C:\Users\lenovo\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "C:\Users\lenovo\anaconda3\Lib\site-packages\selenium\webdriver\common\service.py", line 126, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "C:\Users\lenovo\anaconda3\Lib\urllib\request.py", line 216, in urlopen
    return opener.open(url, data, timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lenovo\anaconda3\Lib\urllib\request.py", line 519, in open
    response = self._open(req, data)
               ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lenovo\anaconda3\Lib\urllib\request.py", line 536, in _open
    result = self._call_chain(self.handl